In [1]:
from __future__ import print_function, division
import scipy


import keras
from keras.datasets import mnist
#from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from data_loader import DataLoader
import numpy as np
import os
from glob import glob

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: TITAN X (Pascal) (0000:01:00.0)


In [2]:


class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

        batch_images = np.random.choice(path, size=batch_size)

        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                img = scipy.misc.imresize(img, self.img_res)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = scipy.misc.imresize(img, self.img_res)
            img=img.transpose(2,0,1)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('./datasets/%s/%sA/*' % (self.dataset_name, data_type))
        path_B = glob('./datasets/%s/%sB/*' % (self.dataset_name, data_type))

        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)
                
                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)
                        
                img_A=img_A.transpose(2, 0, 1)
                img_B=img_B.transpose(2, 0, 1)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        #print(img.shape)
        return img[np.newaxis, :, :, :]

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)


class CycleGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.channels, self.img_rows, self.img_cols)

        # Configure data loader
        self.dataset_name = 'apple2orange'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (1, patch, patch)

        # Number of filters in the first layer of G and D
        self.gf = 32
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id ],
                            optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
         #   d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
          #  u = InstanceNormalization()(u)
            u = Concatenate(axis=1)([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)

        # Upsampling
        u1 = deconv2d(d4, d3, self.gf*4)
        u2 = deconv2d(u1, d2, self.gf*2)
        u3 = deconv2d(u2, d1, self.gf)

        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
           # if normalization:
              #  d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        self.g_AB.summary()
        self.g_BA.summary()
        self.d_A.summary()
        self.d_B.summary()
        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------
               # print(imgs_A.shape)

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)
                #print(fake_A.shape)
                #print(fake_B.shape)
                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch_i, self.data_loader.n_batches,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)

    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

        # Demo (for GIF)
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                
                axs[i,j].imshow(gen_imgs[cnt].transpose(1,2,0))
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

In [4]:
if __name__ == '__main__':
    gan = CycleGAN()
    gan.train(epochs=20, batch_size=1, sample_interval=5)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 3, 128, 128)  0                                            
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 32, 64, 64)   1568        input_9[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_25 (LeakyReLU)      (None, 32, 64, 64)   0           conv2d_37[0][0]                  
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 64, 32, 32)   32832       leaky_re_lu_25[0][0]             
__________________________________________________________________________________________________
leaky_re_l

Total params: 1,545,411
Trainable params: 1,545,411
Non-trainable params: 0
__________________________________________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 3, 128, 128)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 64, 64, 64)        3136      
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 128, 32, 32)       131200    
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 128, 32, 32)       0         
_________________________________________________________________
conv2d_29 (Conv2D)           (Non

/home/emilybla/miniconda2/envs/py3keras/lib/python3.6/site-packages/ipykernel_launcher.py:78: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/emilybla/miniconda2/envs/py3keras/lib/python3.6/site-packages/ipykernel_launcher.py:52: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/emilybla/miniconda2/envs/py3keras/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


[Epoch 0/20] [Batch 0/995] [D loss: 0.491988, acc:  12%] [G loss: 13.144844, adv: 0.845790, recon: 0.521943, id: 0.481205] time: 0:00:47.824295 


/home/emilybla/miniconda2/envs/py3keras/lib/python3.6/site-packages/ipykernel_launcher.py:23: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


...

[Epoch 0/20] [Batch 174/995] [D loss: 0.278575, acc:  40%] [G loss: 5.430655, adv: 0.618287, recon: 0.147163, id: 0.629267] time: 0:01:06.100811 
[Epoch 0/20] [Batch 175/995] [D loss: 0.346304, acc:  40%] [G loss: 5.091297, adv: 0.438225, recon: 0.145103, id: 0.843279] time: 0:01:06.193180 
[Epoch 0/20] [Batch 176/995] [D loss: 0.280026, acc:  68%] [G loss: 5.698108, adv: 0.556920, recon: 0.127600, id: 0.808496] time: 0:01:06.375129 
[Epoch 0/20] [Batch 177/995] [D loss: 0.317845, acc:  46%] [G loss: 5.277506, adv: 0.493715, recon: 0.151866, id: 0.726592] time: 0:01:06.459324 
[Epoch 0/20] [Batch 178/995] [D loss: 0.302938, acc:  15%] [G loss: 4.861232, adv: 0.432028, recon: 0.133325, id: 1.003439] time: 0:01:06.542866 
[Epoch 0/20] [Batch 179/995] [D loss: 0.321411, acc:   0%] [G loss: 5.630019, adv: 0.391138, recon: 0.145075, id: 1.226325] time: 0:01:06.630448 
[Epoch 0/20] [Batch 180/995] [D loss: 0.223599, acc:   0%] [G loss: 5.369964, adv: 0.424435, recon: 0.148566, id: 1.143613] 

[Epoch 0/20] [Batch 231/995] [D loss: 0.277272, acc:  31%] [G loss: 4.976041, adv: 0.449962, recon: 0.119817, id: 0.872456] time: 0:01:12.484115 
[Epoch 0/20] [Batch 232/995] [D loss: 0.329484, acc:  59%] [G loss: 5.251720, adv: 0.368301, recon: 0.143165, id: 0.613560] time: 0:01:12.571230 
[Epoch 0/20] [Batch 233/995] [D loss: 0.294035, acc:  46%] [G loss: 4.445196, adv: 0.386452, recon: 0.085674, id: 1.109814] time: 0:01:12.665290 
[Epoch 0/20] [Batch 234/995] [D loss: 0.269209, acc:  34%] [G loss: 4.893779, adv: 0.408886, recon: 0.134749, id: 0.910583] time: 0:01:12.752549 
[Epoch 0/20] [Batch 235/995] [D loss: 0.248414, acc:  34%] [G loss: 5.768863, adv: 0.377159, recon: 0.128561, id: 1.310859] time: 0:01:12.851505 
[Epoch 0/20] [Batch 236/995] [D loss: 0.261475, acc:   0%] [G loss: 5.322168, adv: 0.402790, recon: 0.121187, id: 1.222763] time: 0:01:13.048701 
[Epoch 0/20] [Batch 237/995] [D loss: 0.261950, acc:   0%] [G loss: 5.196350, adv: 0.462223, recon: 0.150260, id: 0.794525] 

[Epoch 0/20] [Batch 289/995] [D loss: 0.229492, acc:  18%] [G loss: 5.586353, adv: 0.466579, recon: 0.133972, id: 0.776365] time: 0:01:18.773005 
[Epoch 0/20] [Batch 290/995] [D loss: 0.421575, acc:   3%] [G loss: 5.102785, adv: 0.313520, recon: 0.113964, id: 1.193848] time: 0:01:18.847934 
[Epoch 0/20] [Batch 291/995] [D loss: 0.131166, acc:  21%] [G loss: 6.242320, adv: 0.508712, recon: 0.142470, id: 1.689168] time: 0:01:19.016279 
[Epoch 0/20] [Batch 292/995] [D loss: 0.305646, acc:   3%] [G loss: 5.803340, adv: 0.336483, recon: 0.156038, id: 0.985327] time: 0:01:19.106040 
[Epoch 0/20] [Batch 293/995] [D loss: 0.225249, acc:  18%] [G loss: 5.152165, adv: 0.468699, recon: 0.129977, id: 0.875805] time: 0:01:19.190171 
[Epoch 0/20] [Batch 294/995] [D loss: 0.398324, acc:  25%] [G loss: 5.283312, adv: 0.334026, recon: 0.151122, id: 0.948904] time: 0:01:19.276275 
[Epoch 0/20] [Batch 295/995] [D loss: 0.268161, acc:  28%] [G loss: 4.642446, adv: 0.359493, recon: 0.129394, id: 0.624290] 

[Epoch 0/20] [Batch 346/995] [D loss: 0.298574, acc:  40%] [G loss: 4.820464, adv: 0.388820, recon: 0.089262, id: 1.193200] time: 0:01:24.765742 
[Epoch 0/20] [Batch 347/995] [D loss: 0.253737, acc:   9%] [G loss: 4.546799, adv: 0.398378, recon: 0.109254, id: 1.181381] time: 0:01:24.860804 
[Epoch 0/20] [Batch 348/995] [D loss: 0.240115, acc:  31%] [G loss: 4.756771, adv: 0.383646, recon: 0.121950, id: 0.939813] time: 0:01:24.944361 
[Epoch 0/20] [Batch 349/995] [D loss: 0.258652, acc:  21%] [G loss: 4.691925, adv: 0.293111, recon: 0.113840, id: 1.247163] time: 0:01:25.028227 
[Epoch 0/20] [Batch 350/995] [D loss: 0.245525, acc:  18%] [G loss: 5.360309, adv: 0.316676, recon: 0.133332, id: 1.131468] time: 0:01:25.116687 
[Epoch 0/20] [Batch 351/995] [D loss: 0.268118, acc:  15%] [G loss: 4.793578, adv: 0.289182, recon: 0.141361, id: 0.938020] time: 0:01:25.288861 
[Epoch 0/20] [Batch 352/995] [D loss: 0.248973, acc:  21%] [G loss: 4.898986, adv: 0.396548, recon: 0.131007, id: 1.009476] 

KeyboardInterrupt: 

In [10]:
keras.backend.backend()

'theano'

In [6]:
scipy.misc.imread

<function numpy.lib.utils._Deprecate.__call__.<locals>.newfunc(*args, **kwds)>

In [7]:
keras.__version__

'2.2.4'

In [8]:
import tensorflow as tf

In [9]:
tf.__version__

'1.0.1'

In [ ]:
#transpose images to 